# Data Mining Versuch Analyse Globaler Gesundheitsdaten

* Autor: Prof. Dr. Johannes Maucher
* Datum: 18.10.2018

**Abzugeben ist das Jupyter Notebook mit dem verlangten Implementierungen, den entsprechenden Ausgaben, Antworten und Diskussionen/Beschreibungen. Das Notebook ist als .ipynb und als .html abzugeben.**

* [Übersicht Jupyter Notebooks im Data Mining Praktikum](Data%20Mining%20Praktikum.ipynb)


# Einführung
## Lernziele:
In diesem Versuch sollen Kenntnisse in folgenden Themen vermittelt werden:

* Datenimport und Datenexport von und zu 
    * Pandas Dataframes
    * PostgreSQL Datenbanken
* Explorative Datenanalysen (EDA)
* Interaktive Datenvisualisierung mit Bokeh
* Dimensionsreduktion mit PCA und TSNE
* Überwachtes Lernen eines Regressionsmodells
* Unüberwachtes Lernen mit K-Means
* Zeitreihen-Visualisierung

## Vorbereitung

### Datenbankzugriff
1. Installieren Sie PostgreSQL. Mit PostgreSQL sollte auch pgAdmin installiert werden. PgAdmin ist eine open-source Software für die Entwicklung und die Administration von PostgreSQL Datenbanken.
2. Legen Sie über pgAdmin eine Datenbank für das Datamining-Praktikum an. In diese Datenbank werden alle in diesem Versuch relevanten Tabellen geschrieben.
3. Für den Datenbankzugriff aus Python heraus wird in diesem Versuch [SQLAlchemy](http://docs.sqlalchemy.org/en/latest/intro.html) eingesetzt. Machen Sie sich mit den Basics von SQLAlchemy vertraut, z.B. mithilfe von [https://gitlab.mi.hdm-stuttgart.de/maucher/DataScienceProgramming/blob/master/Python/Lecture/07DataBasePandas.ipynb](https://gitlab.mi.hdm-stuttgart.de/maucher/DataScienceProgramming/blob/master/Python/Lecture/07DataBasePandas.ipynb), Abschnitt *Using SQLAlchemy and Pandas*.

### Pandas Dataframe

Machen Sie sich mit den Grundlagen von Pandas vertraut.

### Visualisierung mit Bokeh

Machen Sie sich mit den Grundlagen von [Bokeh](https://bokeh.pydata.org/en/latest/docs/user_guide/quickstart.html) vertraut.

### Dimensionsreduktion
Machen Sie sich mit der Hauptachsentransformation (PCA) und dem t-SNE Verfahren vertraut.

### Regression

Machen Sie sich mit Linearer Regression und Random Forest Regression vertraut.

### Clustering

Machen Sie sich mit dem k-means Clustering Algorithmus vertraut.


# Durchführung

## Einlesen der Daten aus .csv und Ablage in PostgreSQL
In diesem ersten Teil des Versuchs müssen alle relevanten Daten aus .csv-Files eingelesen und in PostgreSQL-Tabellen abgelegt werden. Alle benötigten .csv Files befinden sich im Verzeichnis `gesundheitsdaten`. Die Daten stammen aus folgenden Quellen:

- allgemeine Daten pro Land und Jahr, wie:
    - Lebenserwartung (https://ourworldindata.org/life-expectancy)
    - BIP pro Kopf (https://ourworldindata.org/economic-growth)
    - Gesundheitsausgaben pro Kopf (https://ourworldindata.org/financing-healthcare)
    - Jahresarbeitsstunden pro Kopf (https://ourworldindata.org/working-hours)
- Ernährungsdaten pro Land und Jahr, wie: (https://ourworldindata.org/diet-compositions)
    - durchschnittliche zugeführte Kcal pro Kopf pro Tag für verschiedene Lebensmittel

### Allgemeine Daten
Das unten gegebene Dictionary `filenames_general` definiert aus welchen Dateien (keys), welche Spalten (values) gebraucht werden. Die Bezeichnung der Spalten in den Files weicht von der Spaltenbezeichnung im Dictionary ab. Die im Dictionary angegebenen Namen sollen aber in diesem Versuch (in den Pandas Dataframes und in den Datenbank-Tabellen) verwendet werden.

In [30]:
filenames_general = {'life-expectancy.csv':['Entity','Code','Year','LifeExpectancy'],
    'gdp-per-capita-worldbank.csv':['Entity','Code','Year','GDPperCapita'],
    'annual-healthcare-expenditure-per-capita.csv':['Entity','Code','Year','AnnualHealthcarExpPerCapita'],
    'annual-working-hours-per-persons-engaged.csv':['Entity','Code','Year','AnnualWorkingHourPerPerson']
}

1. Laden Sie die relevanten Daten dieser Files in einen Pandas Dataframe. Die Spalten des resultierenden Dataframes sind *Entity, Code, Year, LifeExpectancy, GDPperCapita, AnnualHealthcarExpPerCapita, AnnualWorkingHourPerPerson*. Als Keys sind *Code*, *Entity* und *Year* zu verwenden. Zeilen in denen mindestens einer dieser Keys fehlt, sind im Dataframe und in der Datenbank zu ignorieren.

> Tipp: Verwenden Sie für das Zusammenführen der Daten aus verschiedenen Files die Methode `merge()` des Pandas-Dataframes.

2. Zeigen Sie für den angelegten Dataframe 
    * die ersten 10 Zeilen
    * die Größe (Anzahl Zeilen und Anzahl Spalten)
    * die Anzahl der NaNs pro Spalte
    
an. 

3. Schreiben Sie den angelegten Dataframe mit der Pandas Methode `to_sql()` in eine Datenbanktabelle mit dem Namen `general_information`.

In [31]:
# All necessary imports 
import psycopg2 #provides drivers for PostgreSQL
import numpy as np
import json #required to access json file
import pandas as pd

In [32]:
# The code reads the contents of the .json file into a Python dictionary.
with open('../db/configLocalP1.json') as f:
    conf = json.load(f)


In [33]:
# Create connection string to connect to PostgreSQL database
from sqlalchemy import create_engine
conn_str ='postgresql://{}:{}@localhost:{}/{}'.format(conf['user'], conf['passw'], conf['port'], conf['database'])
engine = create_engine(conn_str)

In [36]:
# Parse data from the provided csv files via Pandas library
result = pd.DataFrame
df = pd.DataFrame
for key in filenames_general:
  csv_file = '../data/gesundheitsdaten_{}'.format(key)
  df = pd.read_csv(csv_file, names=filenames_general[key], header=0)
  df = df.dropna(subset=['Code', 'Year']) # Drop missing values.
  print(df)
  if result.empty:
    result = df
  else:
    result = result.merge(df, how='left', on=['Entity' ,'Code', 'Year'])
    print('Merged result:{}'.format(result))

table_name = 'general_information'
# Check if datatable already exists,then try to
# Write records stored in a DataFrame to SQL database.
if not engine.has_table(table_name):
    try:
        result.to_sql(name=table_name, index=False, con=engine)
    except (MySQLdb.Error, MySQLdb.Warning) as e:
        print('Error {} occured writing to database.'.format(e))
else:
    print('The table {} already esists.'.format(table_name))


            Entity Code  Year  LifeExpectancy
0      Afghanistan  AFG  1950       27.537001
1      Afghanistan  AFG  1951       27.809999
2      Afghanistan  AFG  1952       28.350000
3      Afghanistan  AFG  1953       28.879999
4      Afghanistan  AFG  1954       29.399000
...            ...  ...   ...             ...
17889     Zimbabwe  ZWE  2011       54.799999
17890     Zimbabwe  ZWE  2012       56.515999
17891     Zimbabwe  ZWE  2013       58.053001
17892     Zimbabwe  ZWE  2014       59.360001
17893     Zimbabwe  ZWE  2015       60.397999

[15386 rows x 4 columns]
           Entity Code  Year  GDPperCapita
0     Afghanistan  AFG  2002   1063.635574
1     Afghanistan  AFG  2003   1099.194507
2     Afghanistan  AFG  2004   1062.249360
3     Afghanistan  AFG  2005   1136.123214
4     Afghanistan  AFG  2006   1161.124889
...           ...  ...   ...           ...
6171     Zimbabwe  ZWE  2012   1848.193306
6172     Zimbabwe  ZWE  2013   1901.183463
6173     Zimbabwe  ZWE  2014   1908

In [37]:
# Print 10 values to check 
print(result[:10])

# Number of rows
print('Total number of rows: {}'.format(result.shape[0]))

# Number of columns
print('Total number of columns: {}'.format(result.shape[1]))

print(result.isnull().sum(axis = 0))

        Entity Code  Year  LifeExpectancy  GDPperCapita  \
0  Afghanistan  AFG  1950       27.537001           NaN   
1  Afghanistan  AFG  1951       27.809999           NaN   
2  Afghanistan  AFG  1952       28.350000           NaN   
3  Afghanistan  AFG  1953       28.879999           NaN   
4  Afghanistan  AFG  1954       29.399000           NaN   
5  Afghanistan  AFG  1955       29.907000           NaN   
6  Afghanistan  AFG  1956       30.403999           NaN   
7  Afghanistan  AFG  1957       30.891001           NaN   
8  Afghanistan  AFG  1958       31.367001           NaN   
9  Afghanistan  AFG  1959       31.834000           NaN   

   AnnualHealthcarExpPerCapita  AnnualWorkingHourPerPerson  
0                          NaN                         NaN  
1                          NaN                         NaN  
2                          NaN                         NaN  
3                          NaN                         NaN  
4                          NaN               

In [38]:
# Test the results written into the database
engine.execute("SELECT * FROM general_information").fetchall()

[('Afghanistan', 'AFG', 1950, 27.537001, None, None, None),
 ('Afghanistan', 'AFG', 1951, 27.809998999999998, None, None, None),
 ('Afghanistan', 'AFG', 1952, 28.35, None, None, None),
 ('Afghanistan', 'AFG', 1953, 28.879998999999998, None, None, None),
 ('Afghanistan', 'AFG', 1954, 29.399, None, None, None),
 ('Afghanistan', 'AFG', 1955, 29.906999999999996, None, None, None),
 ('Afghanistan', 'AFG', 1956, 30.403999, None, None, None),
 ('Afghanistan', 'AFG', 1957, 30.891001, None, None, None),
 ('Afghanistan', 'AFG', 1958, 31.367001000000002, None, None, None),
 ('Afghanistan', 'AFG', 1959, 31.834, None, None, None),
 ('Afghanistan', 'AFG', 1960, 32.292, None, None, None),
 ('Afghanistan', 'AFG', 1961, 32.742001, None, None, None),
 ('Afghanistan', 'AFG', 1962, 33.185001, None, None, None),
 ('Afghanistan', 'AFG', 1963, 33.624001, None, None, None),
 ('Afghanistan', 'AFG', 1964, 34.060001, None, None, None),
 ('Afghanistan', 'AFG', 1965, 34.494999, None, None, None),
 ('Afghanistan', 

In [39]:
# Number of NaN values in every column.
result.isnull().sum(axis = 0)

Entity                             0
Code                               0
Year                               0
LifeExpectancy                     0
GDPperCapita                   10783
AnnualHealthcarExpPerCapita    11811
AnnualWorkingHourPerPerson     12067
dtype: int64

### Ernährungsdaten
Lesen Sie nun gleich wie oben die Ernährungsdaten ein. Die relevanten Spalten der entsprechenden Files sind nun im Dictionary `filenames_nutrition` definiert. Diese Daten sind in einer Datenbanktabelle `nutrition_information` anzulegen.

In [40]:
filenames_nutrition = {
    'life-expectancy.csv':['Entity','Code','Year','LifeExpectancy'],
    'fruit-consumption-per-capita-kilograms-per-year.csv':
            ['Entity','Code','Year','AnnualFruitConsumptionPerCapita'],
    'vegetable-consumption-per-capita-kilograms-per-year.csv':
            ['Entity','Code','Year','AnnualVegetableConsumptionPerCapita'],
    'dietary-compositions-by-commodity-group-1961-2013.csv':
            ['Entity','Code','Year','KcalOther','KcalSugar',
            'KcalOilsFats','KcalMeat','KcalDairyEggs',
            'KcalFruitsVegetables','KcalStarchyRoots','KcalPulses',
            'KcalCerealsGrains','KcalAlcoholicBeverages'],
    'daily-caloric-supply-derived-from-carbohydrates-protein-and-fat.csv':
            ['Entity','Code','Year','KcalAnimalProtein',
            'KcalPlantProtein','KcalFat','KcalCarbohydrates'],
    'daily-per-capita-supply-of-calories.csv':['Entity','Code','Year','DailyCaloriesPerCapita']
}

In [41]:
# Parse data from the provided csv files via Pandas library
result = pd.DataFrame
df = pd.DataFrame
for key in filenames_nutrition:
  csv_file = '../data/gesundheitsdaten_{}'.format(key)
  df = pd.read_csv(csv_file, names=filenames_nutrition[key], header=0)
  df = df.dropna(subset=['Code', 'Year']) # Drop missing values.
  print(df)
  if result.empty:
    result = df
  else:
    result = result.merge(df, how='left', on=['Entity' ,'Code', 'Year'])
    print('Merged result:{}'.format(result))

table_name = 'nutrition_information'
# Check if datatable already exists,then try to
# Write records stored in a DataFrame to SQL database.
if not engine.has_table(table_name):
    try:
        result.to_sql(name=table_name, index=False, con=engine)
    except (MySQLdb.Error, MySQLdb.Warning) as e:
        print('Error {} occured writing to database.'.format(e))
else:
    print('The table {} already esists.'.format(table_name))


            Entity Code  Year  LifeExpectancy
0      Afghanistan  AFG  1950       27.537001
1      Afghanistan  AFG  1951       27.809999
2      Afghanistan  AFG  1952       28.350000
3      Afghanistan  AFG  1953       28.879999
4      Afghanistan  AFG  1954       29.399000
...            ...  ...   ...             ...
17889     Zimbabwe  ZWE  2011       54.799999
17890     Zimbabwe  ZWE  2012       56.515999
17891     Zimbabwe  ZWE  2013       58.053001
17892     Zimbabwe  ZWE  2014       59.360001
17893     Zimbabwe  ZWE  2015       60.397999

[15386 rows x 4 columns]
            Entity Code  Year  AnnualFruitConsumptionPerCapita
0      Afghanistan  AFG  1961                            41.13
1      Afghanistan  AFG  1962                            38.00
2      Afghanistan  AFG  1963                            38.25
3      Afghanistan  AFG  1964                            47.92
4      Afghanistan  AFG  1965                            48.69
...            ...  ...   ...               

### Exemplarische Datenbankabfragen

1. Lesen Sie die Datenbanktabelle `general_information` in einen Pandas Dataframe.
2. Lesen Sie die Spalten `Year`und `AnnualWorkingHourPerPerson` für alle zu `Germany` gehörende Zeilen aus der Tabelle `general_information` in einen Pandas Dataframe.

> **Anmerkung:** Für diesen Versuch kommen relativ kleine Datenmengen zum Einsatz. In diesem Fall kann direkt auf den Pandas Dataframes gearbeitet werden. Für sehr große Datenmengen bietet es sich an, nicht alle Daten in den Arbeitsspeicher zu laden, sondern nur die aktuell benötigten durch entsprechende Datenbankabfragen. 

In [44]:
# Test the PostgrSQL database for specific entries
query="""SELECT "Entity","Code", "Year","AnnualWorkingHourPerPerson" FROM general_information WHERE "Entity"= 'Germany' """
gG=pd.read_sql_query(query,engine)

print(gG)

     Entity Code  Year  AnnualWorkingHourPerPerson
0   Germany  DEU  1875                         NaN
1   Germany  DEU  1885                         NaN
2   Germany  DEU  1895                         NaN
3   Germany  DEU  1905                         NaN
4   Germany  DEU  1911                         NaN
..      ...  ...   ...                         ...
70  Germany  DEU  2011                 1393.120038
71  Germany  DEU  2012                 1374.229772
72  Germany  DEU  2013                 1362.503252
73  Germany  DEU  2014                 1371.100896
74  Germany  DEU  2015                         NaN

[75 rows x 4 columns]


## Geographische Visualisierung der Lebenserwartung

In diesem Teilversuch soll die Lebenserwartung pro Land in einer Weltkarte visualisiert werden. Zu erstellen ist ein Plot der unten dargestellten Art:

![mapLifeExpext](Bilder/mapLifeExpect.png)

### Laden der länderspezifischen Polygondaten

1. Die Lebenserwartung pro Land soll auf einer Karte abgebildet werden. Hierfür werden zunächst die Länderumrisse benötigt, um diese plotten zu können. Die Daten dazu findet man z.B. hier: https://rawgit.com/johan/world.geo.json/master/countries.geo.json. Geodaten für die Länderumrisse können mithilfe der Python Library [geopandas](http://geopandas.org) wie folgt eingelesen werden.

In [46]:
import geopandas as gpd

# Path to the geographical data stored in a .json file
PATH = '../data/json/gesundheitsdaten_'
# Load data into GeoDataFrame
countries = gpd.read_file(PATH + 'countries_geo.json')

In [47]:
# Examine country GeoDataFrame loaded from .json file 
countries.head()

,id,name,geometry
0,AFG,Afghanistan,"POLYGON ((61.21082 35.65007, 62.23065 35.27066..."
1,AGO,Angola,"MULTIPOLYGON (((16.32653 -5.87747, 16.57318 -6..."
2,ALB,Albania,"POLYGON ((20.59025 41.85540, 20.46317 41.51509..."
3,ARE,United Arab Emirates,"POLYGON ((51.57952 24.24550, 51.75744 24.29407..."
4,ARG,Argentina,"MULTIPOLYGON (((-65.50000 -55.20000, -66.45000..."


In [48]:
countries.plot()

<bound method GeoDataFrame.plot of       id                  name  \
0    AFG           Afghanistan   
1    AGO                Angola   
2    ALB               Albania   
3    ARE  United Arab Emirates   
4    ARG             Argentina   
..   ...                   ...   
175  PSE             West Bank   
176  YEM                 Yemen   
177  ZAF          South Africa   
178  ZMB                Zambia   
179  ZWE              Zimbabwe   

                                              geometry  
0    POLYGON ((61.21082 35.65007, 62.23065 35.27066...  
1    MULTIPOLYGON (((16.32653 -5.87747, 16.57318 -6...  
2    POLYGON ((20.59025 41.85540, 20.46317 41.51509...  
3    POLYGON ((51.57952 24.24550, 51.75744 24.29407...  
4    MULTIPOLYGON (((-65.50000 -55.20000, -66.45000...  
..                                                 ...  
175  POLYGON ((35.54566 32.39399, 35.54525 31.78251...  
176  POLYGON ((53.10857 16.65105, 52.38521 16.38241...  
177  POLYGON ((31.52100 -29.25739, 31.32556

1. Da in diesem Teilversuch nur die Lebenserwartung in 2015 betrachtet werden, sind nur diese Zeilen aus dem angelegten Dataframe zu berücksichtigen.
2. Vereinigen Sie die Polygondaten aus dem Dataframe `countries` mit dem Dataframe `general_information`. Achten Sie hierbei, dass Sie den *pandas dataframe* in den *geopandas dataframe* mergen, das Resultat also ein *geopandas dataframe* mergen ist (andernfalls funktioniert der nächste Schritt nicht). 

In [52]:
# Query database to retrieve the data for life expectancy in the year 2015
query = """SELECT "Entity", "Code", "Year","LifeExpectancy" FROM general_information WHERE "Year"= '2015'"""
general_information_2015 = pd.read_sql_query(query,engine)

# Adjust names for the values of "Entity" and "Code" to "name" and "id"
general_information_2015.rename(columns={"Entity":"name", "Code":"id"}, inplace=True)
# Check the new values
print ('The new name-values are \n{}'.format(general_information_2015))

# Merge the two dataframes into a GeoDataFrame
df = gpd.GeoDataFrame
df = countries.merge(general_information_2015, how='inner',on=["name", "id"])
print('The merged GeoDataFrame values are \n{}'.format(df))

The new name-values are 
                    name        id  Year  LifeExpectancy
0            Afghanistan       AFG  2015       63.287998
1                Albania       ALB  2015       78.174004
2                Algeria       DZA  2015       75.860001
3                 Angola       AGO  2015       61.241001
4    Antigua and Barbuda       ATG  2015       76.207001
..                   ...       ...   ...             ...
198       Western Sahara       ESH  2015       69.212997
199                World  OWID_WRL  2015       71.429001
200                Yemen       YEM  2015       64.742996
201               Zambia       ZMB  2015       61.396999
202             Zimbabwe       ZWE  2015       60.397999

[203 rows x 4 columns]
The merged GeoDataFrame values are 
      id                  name  \
0    AFG           Afghanistan   
1    AGO                Angola   
2    ALB               Albania   
3    ARE  United Arab Emirates   
4    ARG             Argentina   
..   ...                   


5. Durch Aufruf der Methode `to_json()` kann ein Dataframe in json transformiert werden. Die von der Methode zurückgegebenen Daten können einem [Bokeh GeoJSONDataSource](https://bokeh.pydata.org/en/latest/docs/user_guide/geo.html#geojson-data)-Objekt beim Anlegen übergeben werden. Das *GeoJSONDataSource*-Objekt kann direkt der entsprechenden Bokeh-plotting Funktion übergeben werden. Erzeugen Sie mit diesen Hinweisen einen interaktiven Weltkartenplot der oben dargestellten Art. Beim *Mouse-Over* über ein Land soll der Name des Landes und die Lebenserwartung angezeigt werden.
> Alternative: Eine einfachere Lösung für die Geovisualisierung bietet [geopandas](http://geopandas.org/mapping.html). Allerdings sind damit keine interaktiven Elemente realisierbar. 
6. Diskutieren Sie die aus der Visualisierung gewonnenen Erkenntnisse.

In [55]:
# Write the GeoPandaFrame to GeoJSON file format
df.to_file("../data/json/lifeexp_geodata_2015.json", driver="GeoJSON")

In [ ]:
import json

from bokeh.io import output_file, show
from bokeh.models import GeoJSONDataSource
from bokeh.plotting import figure

## Korrelation der Lebenserwartung mit volkswirtschaftlichen Metriken
In diesem Abschnitt soll die paarweise Korrelation von 
* `GDPperCapita`
* `AnnualHealthcarExpPerCapita`

mit der `LifeExpectancy` untersucht werden. 

1. Für die visuelle Korrelationsanalyse erzeugen Sie je einen Scatterplot, in dem `LifeExpectancy` über `GDPperCapita` bzw. über `AnnualHealthcarExpPerCapita` dargestellt ist.
2. Berechnen Sie mit der `numpy`-Funktion `corrcoef` die beiden paarweisen Korrelationen. 
3. Diskutieren Sie das Ergebnis.

## Korrelation der Lebenserwartung mit Ernährungsdaten
Untersuchen Sie dann so wie im vorigen Abschnitt wie die einzelnen Nuitrition-Merkmale mit der `LifeExpectancy` korrelieren. Diskutieren Sie das Ergebnis.

## Time Series Analysis

Auf der Basis des Dataframes `nutrition_information`: Stellen Sie die zeitliche Entwicklung des Merkmals
1. `KcalFat` 
2. `DailyCaloriesPerCapita` 

von 1995 bis 2015 für die Länder Bolivia, Bulgaria, Egypt, Kenya, United States, China, Brazil, Germany graphisch dar. Erzeugen Sie hierfür pro Merkmal einen Plot, in dem die Linegraphs aller genannten Länder dargestellt sind. 

Interpretieren Sie diese Darstellungen

## Jahr mit den meisten non-Nans
1. Bestimmen Sie das Jahr, für das am meisten non-Nans in dem in der vorigen Teilaufgabe erstellten Dataframe `nutrition_information` existieren.
2. Erstellen Sie einen neuen Dataframe, der nur die non-Nan Daten für das zuvor bestimmte Jahr enthält.
3. Mergen Sie diesen neuen Dataframe mit dem Dataframe `general_information` ohne die Spalte `AnnualWorkingHourPerPerson`. Der so gebildete Dataframe wird im Weiteren mit `data_nut_gen` bezeichnet.

## Dimensionsreduktion mit PCA und TSNE

Der oben konstruierte Dataframe `data_nut_gen` sollte ausschließlich non-NAN Werte haben und Daten nur eines Jahres enthalten. Alle Spalten von `data_nut_gen`, außer `Entity`, `Code`, `Year` und `LifeExpectancy` werden im Folgenden als *Merkmalsspalten* bezeichnet.

1. Führen Sie auf dem numpy-Array, das nur die Werte der *Merkmalspalten* enthält, eine Principal Component Analysis mit dem entsprechenden [scikit-learn Modul PCA](http://scikit-learn.org/stable/modules/generated/sklearn.decomposition.PCA.html) durch. Bestimmen Sie die zwei Hauptachsen. 
2. Bestimmen Sie für das trainierte Modell den `explained_variance_ratio`. Was sagen diese Zahlen aus?
2. Stellen Sie die auf die 2 Hauptachsen transformierten Daten der Merkmalsspalten von `data_nut_gen` in einem Bokeh-Plot dar. In diesem soll die Farbe der Punkte durch die Werte der Spalte `LifeExpectancy` codiert werden.
3. Führen Sie nun eine Dimensionalitätsreduktion mit dem [scikit-learn Modul TSNE](http://scikit-learn.org/stable/modules/generated/sklearn.manifold.TSNE.html) auf 2 Achsen durch. Visualisieren Sie auch dieses Resultat mit einem Bokeh-Plot (wie in Teilaufgabe 2).
4. Diskutieren Sie das Ergebnis der beiden Dimensionsreduktionen.

## Regression
Im Folgenden soll aus den Merkmalsspalten das Dataframes `data_nut_gen` die Lebenserwartung (Spalte `LifeExpectancy`) vorhergesagt werden.
1. Partitionieren Sie die Daten mit der [scikit-learn Methode](http://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html), so dass $2/3$ der Daten für das Training und der Rest für den Test verwendet werden.
2. Trainieren Sie [ein lineares Regressionsmodell](http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LinearRegression.html) mit den Trainingsdaten und bestimmen Sie den *Mean-Absolute-Error* und den *R2*-Score auf Trainings- und Testdaten.
3. Wie kann mit dem trainierten Linear-Regression Modell auf die Bedeutung (Wichtigkeit) der verwendeten Merkmale geschlossen werden. 
3. Trainieren Sie nun einen [Random Forest Regressor](http://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestRegressor.html) und bestimmen Sie wieder *Mean-Absolute-Error* und den *R2*-Score auf Trainings- und Testdaten.
3. Wie kann mit dem trainierten Random-Forest Modell auf die Bedeutung (Wichtigkeit) der verwendeten Merkmale geschlossen werden. 
4. Was ist der *R2-Score*?
4. Diskutieren Sie die Ergebnisse

In [0]:
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score,mean_absolute_error

## Clustering der Länder nach Ihrem Pro-Kopf-Kalorienverbrauch
1. Bestimmen Sie zunächst alle Länder für die das Merkmal `DailyCaloriesPerCapita` im Dataframe `nutrition_information` für alle Jahre von 1960 bis 2012 einen Wert hat. Konstruieren Sie dann ein numpy-Array $X$, in dem das Element $C_{i,j}$ in Zeile $i$, Spalte $j$, den `DailyCaloriesPerCapita`-Wert des Landes $i$ im $j.ten$ Jahr zwischen 1960 und 2012 enthält.
2. Clustern Sie die Länder nach ihrer zeitlichen Entwicklung des `DailyCaloriesPerCapita`-Wertes - also die Zeilen des Arrays $X$. Verwenden Sie hierfür den [K-Means Algorithmus von scikit-learn](http://scikit-learn.org/stable/modules/generated/sklearn.cluster.KMeans.html). Stellen Sie dabei die Clusteranzahl auf $k=5$ ein. 
3. Für jedes der $k=5$ Cluster: Erzeugen Sie einen Plot, in dem der zeitliche Verlauf des `DailyCaloriesPerCapita`-Wertes aller zum jeweiligen Cluster gehörenden Länder als Line-Graph dargestellt ist.
4. Charakterisieren Sie die 5 Cluster anhand der clusterspezifischen Visualisierung.
    

In [0]:
from sklearn.cluster import KMeans